# AI Cyoda configurations Q&A with RAG Langchain

Welcome to this Jupyter notebook! This notebook serves as your guide to developing an AI-powered Question & Answer system using the Langchain library. This system utilizes the Retrieval-Augmented Generation (RAG) model that leverages OpenAI's GPT-3 model to provide intelligent and context-aware responses.

The primary purpose of this notebook is to generate Cyoda related code. It does so by interacting with the data set available in the official Cyoda repository. 

## What will we cover?

In this notebook, we will go through the following steps:

1. **Setting up the environment**: We will install necessary libraries and load environment variables.

2. **Initializing the AI model**: We will initialize the ChatOpenAI model with the appropriate parameters.

3. **Loading instructions and entities**: We will load instructions and entities from the official repository using the GitLoader.

4. **Splitting documents and creating a vectorstore**: We will split the loaded documents into chunks and create a vectorstore using the Chroma library.

5. **Defining prompts for contextualizing and answering questions**: We will define prompts that the AI model will use to contextualize and answer questions.

6. **Creating a retrieval chain**: We will create a retrieval chain that combines the history-aware retriever and the question-answer chain.

7. **Running the chatbot**: Finally, we will run the chatbot and see it in action!

## Let's get started!

Please follow along with the code cells and comments to understand each step of the process. If you have any questions or run into any issues, feel free to ask for help. Happy coding!

Install requirements

In [ ]:
pip install -r ../requirements.txt

### Load environment variables

In [20]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
WORK_DIR = os.environ["WORK_DIR"]

In [ ]:
##for google colab (optional)
# This cell is optional and can be skipped
# from google.colab import userdata
# API_KEY = userdata.get('OPENAI_API_KEY')

### Handle unsupported version of sqlite3 (optional)

In [ ]:
pip install pysqlite3-binary

In [ ]:
import sys

__import__("pysqlite3")
sys.modules["sqlite3"] = sys.modules["pysqlite3"]

### Initialize ChatOpenAI

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import GitLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import HumanMessage

In [22]:
llm = ChatOpenAI(
    temperature=0.7,
    max_tokens=6000,
    model="gpt-3.5-turbo-16k",
    openai_api_key=OPENAI_API_KEY,
)

### Load instructions and entities from the official cyoda repository

In [23]:
loader = GitLoader(
    clone_url="https://github.com/Cyoda-platform/cyoda-ai",
    repo_path=WORK_DIR,
    branch="cyoda-ai-configurations-3.0.x",
    file_filter=lambda file_path: file_path.startswith(f"{WORK_DIR}/data/code/"),
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

Number of documents loaded: 11


### Split documents and create vectorstore

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [25]:
count = vectorstore._collection.count()
print(count)

40


### Define prompts for contextualizing question and answering question

In [26]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [27]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question

In [28]:
qa_system_prompt = """You are a code assistant. You are aware of grpc client CyodaCalculationMemberClient. \
CalculationMemberGreetEvent.json, CalculationMemberJoinEvent.json, CloudEventType.json, EntityProcessorCalculationRequest.json, EntityProcessorCalculationResponse.json are used for schema generation. \
Also use available proto files.You should write code in different languages to produce similar grpc clients.. \
You should do your best to answer the question. \
Use the following pieces of retrieved context to answer the question. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

### Create retrieval chain

In [29]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Initialize chat history and relevant functions

In [30]:
chat_history = {}

In [31]:
# Function to add a message to the chat history
def add_to_chat_history(id, question, message):
    if id in chat_history:
        chat_history[id].extend([HumanMessage(content=question), message])
    else:
        chat_history[id] = [HumanMessage(content=question), message]

In [32]:
# Function to clear chat history
def clear_chat_history(id):
    if id in chat_history:
        del chat_history[id]

In [33]:
def ask_question(id, question):
    ai_msg = rag_chain.invoke(
        {"input": question, "chat_history": chat_history.get(id, [])}
    )
    add_to_chat_history(id, question, ai_msg["answer"])
    return ai_msg["answer"]

### Start a chat session

In [34]:
import uuid

# Generate a unique ID for the chat session
id = uuid.uuid1()

In [39]:
##Rewrite CyodaCalculationMemberClient to java. Produce ready to use code. Leave spring boot
question = "Rewrite CyodaCalculationMemberClient to java. Produce ready to use code. Leave Spring Boot related syntax."
result = ask_question(id, question)
print(result)

```java
import com.fasterxml.jackson.databind.ObjectMapper;
import io.cloudevents.core.builder.CloudEventBuilder;
import io.cloudevents.core.data.PojoCloudEventData;
import io.cloudevents.core.format.EventFormat;
import io.cloudevents.core.provider.EventFormatProvider;
import io.cloudevents.protobuf.ProtobufFormat;
import io.cloudevents.v1.proto.CloudEvent;
import io.grpc.ManagedChannel;
import io.grpc.ManagedChannelBuilder;
import io.grpc.stub.StreamObserver;
import org.cyoda.cloud.api.event.common.BaseEvent;
import org.cyoda.cloud.api.event.common.CloudEventType;
import org.cyoda.cloud.api.event.processing.CalculationMemberJoinEvent;
import org.cyoda.cloud.api.event.processing.EntityProcessorCalculationRequest;
import org.cyoda.cloud.api.event.processing.EntityProcessorCalculationResponse;

import java.util.UUID;
import java.util.concurrent.TimeUnit;

public class CyodaCalculationMemberClient implements AutoCloseable {

    private final ManagedChannel managedChannel;
    private fin

In [40]:
print(chat_history)

{UUID('082365a0-fbfb-11ee-ac34-f01898ec22af'): [HumanMessage(content='Rewrite CyodaCalculationMemberClient to java. Produce ready to use code. Leave spring boot features.'), '```java\nimport com.fasterxml.jackson.databind.ObjectMapper;\nimport io.cloudevents.core.builder.CloudEventBuilder;\nimport io.cloudevents.core.data.PojoCloudEventData;\nimport io.cloudevents.core.format.EventFormat;\nimport io.cloudevents.core.provider.EventFormatProvider;\nimport io.cloudevents.protobuf.ProtobufFormat;\nimport io.cloudevents.v1.proto.CloudEvent;\nimport io.grpc.ManagedChannel;\nimport io.grpc.ManagedChannelBuilder;\nimport io.grpc.stub.StreamObserver;\nimport org.cyoda.cloud.api.event.common.BaseEvent;\nimport org.cyoda.cloud.api.event.common.CloudEventType;\nimport org.cyoda.cloud.api.event.common.DataPayload;\nimport org.cyoda.cloud.api.event.processing.CalculationMemberJoinEvent;\nimport org.cyoda.cloud.api.event.processing.EntityProcessorCalculationRequest;\nimport org.cyoda.cloud.api.event.

In [41]:
#clear chat history if necessary
clear_chat_history(id)